# 1 - Generación de señales de Hahn
* Programa de generacion y lectura de datos de señales de RMN Hahn
---

Importación de paquetes

In [ ]:
using CSV
using DataFrames
using Plots

Constantes útiles para el cálculo de las señales (Las unidades utilzadas estan en μm, s y T)

In [ ]:
γ = 2.675e8  # Factor girómagetico del esín nuclear del proton (s⁻¹T⁻¹) de https://physics.nist.gov/cgi-bin/cuu/Value?gammap
G = 8.73e-7  # Gradiente externo (T/μm) de Validating NOGSE’s size distribution predictions in yeast cells Paper 1
D0 = 1e3; # Coeficiente de difusión (μm²/s) del Ejercico

Primero para generar las señales tenemos que tener una distribución de tamaños de compartimientos lc

Generación de distribution log-normal, con parámetros lcm y σ, en este caso lcm es el tamaño medio del compartimiento y σ es la desviación estándar del tamaño del compartimiento


In [ ]:
function P(lc, lcm, σ)
    """Función que genera una distribución log-normal
    Parámetros:
        lc: tamaño de compartimiento
        lcm: tamaño medio de compartimiento
        σ: desviación estándar de compartimiento

    Retorna:
        P(lc): distribución log-normal
    """
    return ( exp( -(log(lc) - log(lcm))^2 / (2σ^2) ) ) / (lc*σ*sqrt(2π))
end;

Si el tamaño de los compartimientos es único entonces la señal detectada es simplemente la magnetización de Hahn M(t)

Función M_l Magnetización de Hahn, para un tiempo t y un tamaño lc

In [ ]:
function Ml_Hahn(t, lc)
    """Función que genera la magnetización de Hahn
    Parámetros:
        t: tiempo
        lc: tamaño de compartimiento
    Retorna:
        Ml_Hahn(t, lc): magnetización de Hahn
    """
    τc = lc^2 / (2 * D0) # Tiempo de correlación
    term1 = -γ^2 * G^2 * D0 * τc^2
    term2 = t - τc * (3 + exp(-t / τc) - 4 * exp(-t / (2 * τc)))
    return exp(term1 * term2)
end;

La señal detectada S(t) es la suma de las señales de Hahn ponderadas por la distribución de tamaños de compartimientos P(lc)

Función S_hanh(t) recibe el tamaño medio de compartimiento lcm, la desviación estándar σ, la cantidad de compartimientos N, el tamaño mínimo de compartimiento l0, el tamaño máximo de compartimiento lf y el tiempo t

In [ ]:
function S_han(lcm, σ, N, l0, lf, t)
    """Función que genera la señal detectada S(t)
    Parámetros:
        lcm: tamaño medio de compartimiento
        σ: desviación estándar de compartimiento
        N: cantidad de compartimientos
        l0: tamaño mínimo de compartimiento
        lf: tamaño máximo de compartimiento
        t: tiempo
    Retorna:
        S: señal detectada
    """
    lc = range(l0, lf, length = N) # Generamos los tamaños de compartimientos lc desde l0 hasta lf

    P_lc = P.(lc,lcm, σ) # Consideramos media lcm y ancho σ
    
    M_lc = Ml_Hahn.(t, lc) # Calculamos M_lc(t) para cada tamaño de compartimiento
    
    S = sum(M_lc .* P_lc) # Calculamos la señal de Hahn
    return S
end;

Finalmente la función que genera los datos de las señales noramlizadas y las distribuciones de tamaños de compartimientos

In [ ]:
function GenData(N, lcm, σ, l0, lf, t)
    """Función que genera los datos de las señales normalizadas y las distribuciones de tamaños de compartimientos
    Parámetros:
        N: cantidad de compartimientos lc
        lcm: tamaño medio de compartimiento
        σ: desviación estándar de compartimiento
        l0: tamaño mínimo de compartimiento
        lf: tamaño máximo de compartimiento
        time_sim: tiempo máximo de simulación
        time_sample_lenght: cantidad de puntos de tiempo
    Retorna:
        S: señal detectada
        P_l: distribución de tamaños de compartimientos
    """
    # Generamos los tamaños de compartimientos
    lc = range(l0, lf, length = N)

    # Generamos las distribuciones
    P_l = P.(lc, lcm, σ)

    # Calculamos la señal noramlizadas
    S0 = S_han(lcm, σ, N, l0, lf, 0)
    S = S_han.(lcm, σ, N, l0, lf, t) ./ S0
    return S, P_l
end;

Ahora generamos los datos en CSV para cada combinación de parámetros en el path especificado, este va a ser el mismo que use para leer los datos

In [ ]:
function GenCSVData(N, l0, lf, t, lcms, σs, path)
    """Función que genera los datos en CSV para cada combinación de parámetros en el path especificado
    Parámetros:
        N: cantidad de compartimientos lc
        time_sample_lenght: cantidad de puntos de tiempo
        l0: tamaño mínimo de compartimiento
        lf: tamaño máximo de compartimiento
        t: tiempo
        lcms: vector de tamaños medios de compartimientos
        σs: vector de desviaciones estándar
        path: path donde se guardan los datos
    Retorna:
        CSV: archivos con los datos generados
    """
    # Función auxiliar para llenar con ceros debido a que los vectores de tamaños de compartimientos y señales tienen diferentes longitudes
    function fill_missing(value, column, max_lenght)
        # Si la longitud de la columna es menor que la longitud máxima, llenamos con ceros
        if length(column) < max_lenght
            return vcat(column,fill(value, max_lenght - length(column)))
        else
            return column
        end
    end

    # Generamos los tamaños de compartimientos
    l = range(l0, lf, length = N)
    # Calculamos la longitud máxima entre los tamaños de compartimientos y las señales (o tiempos)
    max_lenght = maximum(length.([l,t]))

    # Para cada combinación de tamaños medios de compartimientos y desviaciones estándar
    for lcm in lcms
        for σ in σs
            # Generamos los datos de las señales normalizadas y las distribuciones de tamaños de compartimientos
            S, P_l = GenData(N, lcm, σ, l0, lf, t)
            # Llenamos con ceros las señales
            S = fill_missing(0, S, max_lenght)
            # Guardamos todos los datos en un DataFrame
            df = DataFrame(S = S, P_l = P_l)
            # Guardamos el DataFrame en un archivo CSV
            CSV.write(path * "/$(lcm)_$(σ)l_$N.csv", df)
        end
    end
end;

Estos datos en CSV contienen 2 columnas la primera es la señal y la segunda la distribución de tamaños de compartimientos
Como los datos pueden tener un muestreo diferente se rellenan con ceros para que tengan la misma longitud
Por esto es importante tener en cuenta el muestreo del tiempo y el de los tamaños de compartimientos

Lectura de los datos que se generaron con los mismos parámetros que GenCSVData
Esta función acumula todos los archivos generados de las señales y las distribuciones de tamaños de compartimientos especificando cada lcm y σ

In [ ]:
function ReadCSVData(N, l0, lf, t, lcms, σs, path)
    """Función que lee los datos generados
    Parámetros:
        N: cantidad de compartimientos lc
        time_sample_lenght: cantidad de puntos de tiempo
        l0: tamaño mínimo de compartimiento
        lf: tamaño máximo de compartimiento
        t: tiempo
        lcms: vector de tamaños medios de compartimientos
        σs: vector de desviaciones estándar
        path: path donde se guardan los datos
    Retorna:
        Probabilitys: distribuciones de tamaños de compartimientos
        Signals: señales generadas
    """
    # Generamos los tamaños de compartimientos
    lc = range(l0, lf, length = N)
    # Calculamos la longitud máxima entre los tamaños de compartimientos y las señales (o tiempos)
    length_t = length(t)
    length_lc = length(lc)
    max_lenght = maximum(length.([t, lc]))

    # Inicializamos los arreglos de distribuciones de tamaños de compartimientos y señales
    Probabilitys = zeros(length(lcms), length(σs), max_lenght)
    Signals = zeros(length(lcms), length(σs), max_lenght)

    # Para cada combinación de tamaños medios de compartimientos y desviaciones estándar
    for lcm in lcms
        for σ in σs
            # Leemos los datos de las señales normalizadas y las distribuciones de tamaños de compartimientos
            df = CSV.read(path * "/$(lcm)_$(σ)l_$N.csv", DataFrame)
            # Encontramos los índices de los tamaños medios de compartimientos y desviaciones estándar
            # para llenar los arreglos de distribuciones de tamaños de compartimientos y señales
            Probabilitys[findall(x -> x == lcm, lcms), findall(x -> x == σ, σs), :] = df.P_l
            Signals[findall(x -> x == lcm, lcms), findall(x -> x == σ, σs), :] = df.S

        end
    end

    return Probabilitys, Signals

end;

# 2 - Generación y guardado de señales de Hahn
* Programa que genera y guarda señales de RMN Hahn en archivos CSV para su posterior lectura
---

## Importación de parámetros
* Los parámetros como el tamaño de los compartimientos, la cantidad de compartimientos, el tiempo de muestreo, el tiempo de adquisición, el tamaño mínimo y máximo de los compartimientos, el tamaño medio de los compartimientos y la desviación estándar del tamaño de los compartimientos son importados desde otro programa

In [ ]:
include("C:\\Users\\Propietario\\Desktop\\ib\\Tesis_V1\\Proyecto_Tesis\\1-GeneracionDeDatos\\Parametros.jl");

Se generan los datos utilizando la función GenCSVData pasando los parámetros necesarios y el path donde se van a guardar los datos

In [ ]:
path = "C:/Users/Propietario/Desktop/ib/Tesis_V1/Proyecto_Tesis/1-GeneracionDeDatos/Datos_Final/datos_crudos"
# GenCSVData(N, time_sample_lenght, l0, lf, t, lcms, σs, path)

Como tenemos los datos en CSV usamos la función de lectura de datos para leerlos y que nos devuelva todas las señales y distribuciones de tamaños de compartimientos
Para esto utilizamos la función ReadCSVData en el mismo path en el que guardamos las señales anteriormene.

In [ ]:
Probabilitys, Signals = ReadCSVData(N, l0, lf, t, lcms, σs, path)

# 3 - Rerordenamiento de datos para PCA
* Programa que reordena los datos de las señales de RMN Hahn para su posterior análisis con PCA

Extraemos datos útiles para realizar este re ordenamiento

In [ ]:
length_σs =  length(σs) # Cantidad de desviaciones estándar
length_lcms = length(lcms) # Cantidad de tamaños medios de compartimientos
length_t = length(t) # Cantidad de puntos de tiempo
max_length = maximum(length.([t, lc])); # Máxima cantidad de puntos de tiempo o tamaños de compartimientos

La matriz de datos que creamos tiene 3 dimensiones (tamaños medios de compartimientos, desviaciones estándar y señales o distribuciones de probabilidad)

Re organizamos los datos de la siguiente manera, tenemos una matriz de tamaño (dim3, dim1*dim2) es decir (señales/distribuciones, tamaños medios de compartimientos * desviaciones estándar)
Los nuevos datos van a ser de la forma, señal/P para lcm 1 y σ 1, señal/P para lcm 2 y σ 1, señal/P para lcm 3 y σ 1, ..., señal/P para lcm 1 y σ 2, señal/P para lcm 2 y σ 2, señal/P para lcm 3 y σ 2, ... Y así hasta terminar con todas las señales y distribuciones

In [ ]:
function reshape_data(old_matrix, old_shape, new_shape)
    """Función que re organiza los datos de entrada y salida en un formato que podemos guardar en un archivo csv
    
    Parámetros:
        old_matrix: matriz de datos
        old_shape: tamaño de la matriz de datos
        new_shape: nuevo tamaño de la matriz de datos
    
    Retorna:
        new_matrix: matriz de datos re organizada
    """
    old_matrix = old_matrix
    dim1, dim2, dim3 = old_shape # La matriz de datos tiene 3 dimensiones (tamaños medios de compartimientos, desviaciones estándar y señales o distribuciones de probabilidad)
    new_matrix = zeros(Float64, dim3, dim1*dim2)

    # Re organizamos los datos de la siguiente manera, tenemos una matriz de tamaño (dim3, dim1*dim2) es decir (puntos de tiempo/distribuciones, tamaños medios de compartimientos * desviaciones estándar)
    for i in 1:dim1
        for j in 1:dim2
            for k in 1:dim3
                new_matrix[k,(i - 1) * dim2 + j] = old_matrix[i,j,k]
            end
        end
    end

    return new_matrix
end;

Nuevo tamaño de los datos

In [ ]:
new_size = (length_σs * length_lcms, max_length)

Ahora si tenemos los datos de entrada y salida es decir las señales y las distribuciones de probabilidad en un formato que podemos guardar en un archivo csv
estos datos son todas las señales generadas en las columnas de una matriz
y para el otro archivo las distribuciones de tamaños de compartimientos en las columnas de una matriz

In [ ]:
dataSignals = reshape_data(Signals, size(Signals), new_size)
dataProbd = reshape_data(Probabilitys, size(Probabilitys), new_size)

En un momento para tener aun DataFrame llenamos los datos de la señal con 0s los sacamos de cada columna.

In [ ]:
dataSignals = dataSignals[1:length_t, :]

Guardamos estos datos en los archivos CSV
* Elegir el path donde se van a guardar los datos

In [ ]:
path_save = "C:\\Users\\Propietario\\Desktop\\ib\\Tesis_V1\\Proyecto_Tesis\\1-GeneracionDeDatos\\Datos_Final\\datos_PCA";

In [ ]:
df_dataSignals = DataFrame(dataSignals, :auto)
df_dataProbd = DataFrame(dataProbd, :auto)

Los datos de las señales se generan en un archivo CSV donde tiene en la primera fila el indice de las señales como x1, x2 ... xn y en cada columna los datos de cada una tiempos quedan guardados en el programa Parametros.jl

In [ ]:
CSV.write(path_save * "\\dataSignals.csv", df_dataSignals)
CSV.write(path_save * "\\dataProbd.csv", df_dataProbd)